# Семинар 11. PyTorch

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

from sklearn.datasets import load_boston
from torch import nn

from tqdm.notebook import tqdm

%matplotlib inline

### 1. Pytorch и numpy 

Многие функции в pytorch очень напоминают numpy :)

In [ ]:
a = np.random.rand(5, 3)
a

In [ ]:
print(f"Размеры: {a.shape}")

In [ ]:
print(f"Добавили 5:\n{a + 5}")

In [ ]:
print(f"Посчитали произведение X X^T:\n{a @ a.T}")

In [ ]:
print(f"Среднее по колонкам:\n{a.mean(axis=-1)}")

In [ ]:
print(f"Изменили размеры: {a.reshape(3, 5).shape}")

Аналогичные операции в **pytorch** выглядят следующим образом:

In [ ]:
x = torch.rand(5, 3)
x

In [ ]:
print(f"Размеры: {x.shape}")

In [ ]:
type(x)

In [ ]:
x.dtype

In [ ]:
print(f"Добавили 5:\n{x + 5}")

In [ ]:
# для перемножения тензоров высокой размерности читайте документацию по различным вариантам:
# torch.mm, torch.matmul, torch.bmm, @
print(f"X X^T  (1):\n{torch.matmul(x, x.transpose(1, 0))}\n")
print(f"X X^T  (2):\n{x.mm(x.t())}")

In [ ]:
print(f"Среднее по колонкам:\n{x.mean(dim=-1)}")

In [ ]:
print(f"Изменили размеры:\n{x.view([3, 5]).shape}\n")

# будьте внимательны и не используйте view для транспонирования осей
print(f"По-другому изменили размеры:\n{x.view_as(x.t()).shape}\n")
print(f"Но не транспонировали!\n{x.view_as(x.t()) == x.t()}")

Небольшой пример того, как меняются операции:

* `x.reshape([1,2,8]) -> x.view(1,2,8)`

* `x.sum(axis=-1) -> x.sum(dim=-1)`

* `x.astype("int64") -> x.type(torch.LongTensor)`

Для помощи вам есть [таблица](https://github.com/torch/torch7/wiki/Torch-for-Numpy-users), которая поможет вам найти аналог операции в numpy


#### Разминка на pytorch

При помощи pytorch посчитайте сумму квадратов натуральных чисел от 1 до 10000.

In [ ]:
# YOUR CODE

### 2. Создаем тензоры в pytorch 

In [ ]:
x = torch.empty(5, 3)  # пустой тензор
print(x)

In [ ]:
x = torch.rand(5, 3)  # случайный тензор
print(x)

In [ ]:
x = torch.zeros(5, 3, dtype=torch.float32)  # тензор нулей с указанием типов чисел
print(x)

In [ ]:
x = torch.tensor([5.5, 3],  dtype=torch.double)  # конструируем тензор из питоновского листа
print(x)

In [ ]:
x1 = x.new_ones(5, 3)  # используем уже созданный тензор для создания тензора из единичек
print(x1) 

In [ ]:
x = torch.randn_like(x1, dtype=torch.float)  # создаем случайный тензор с размерами x
print(x, x.size())

In [ ]:
y = torch.rand(5, 3)
print(x + y)  # операция сложения

In [ ]:
z = torch.add(x, y)  # очередная операция сложения
print(z)

In [ ]:
torch.add(x, y, out=z)  # и наконец последний вид
print(z)

In [ ]:
print(x * y)  # поэлементное умножение

In [ ]:
print(x.unsqueeze(0).shape)  # добавили измерение в начало, аналог броадкастинга

In [ ]:
print(x.unsqueeze(0).unsqueeze(1).squeeze().shape)  # убрали измерение в начале

Мы также можем делать обычные срезы и переводить матрицы назад в numpy:

In [ ]:
a = np.ones((3, 5))
x = torch.ones((3, 5))
print(np.allclose(x.numpy(), a))
print(np.allclose(x.numpy()[:, 1], a[:, 1]))

### 3. Работаем с градиентами 

In [ ]:
boston = load_boston()
plt.scatter(boston.data[:, -1], boston.target);

В pytorch есть возможность при создании тензора указывать нужно ли считать по нему градиент или нет, с помощью параметра `requires_grad`. Когда `requires_grad=True` мы сообщаем фреймворку, о том, что мы хотим следить за всеми тензорами, которые получаются из созданного. Иными словами, у любого тензора, у которого указан данный параметр, будет доступ к цепочке операций и преобразований совершенными с ними. Если эти функции дифференцируемые, то у тензора появляется параметр `.grad`, в котором хранится значение градиента.

Если к результирующему тензору применить метод `.backward()`, то фреймворк посчитает по цепочке градиент для всех тензоров, у которых `requires_grad=True`.

In [ ]:
w = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

In [ ]:
w = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

x = torch.tensor(boston.data[:, -1] / boston.data[:, -1].max(), dtype=torch.float32)
y = torch.tensor(boston.target, dtype=torch.float32)

# только создали тензоры и в них нет градиентов
assert w.grad is None
assert b.grad is None

In [ ]:
y_pred = w * x + b                    # и опять совершаем операции с тензорами
loss = torch.mean((y_pred - y) ** 2)  # совершаем операции с тензорами
loss.backward()                       # считаем градиенты

In [ ]:
# сделали операции и посчитали градиенты
assert w.grad is not None
assert b.grad is not None

print("dL/dw = \n", w.grad)
print("dL/db = \n", b.grad)

Для доступа к значениям в тензоре используйте атрибут `.data`:

In [ ]:
w.data

In [ ]:
from IPython.display import clear_output

for i in range(300):
    y_pred = w * x + b
    # попробуйте сделать полиномиальную регрессию в данном предсказании и посчитать градиенты после
    loss = torch.mean((y_pred - y)**2)
    loss.backward()

    # делаем шаг градиентного спуска с lr = .05
    w.data -=  0.05 * w.grad# YOUR CODE
    b.data -=  0.05 * b.grad# YOUR CODE

    # обнуляем градиенты, чтобы на следующем шаге опять посчитать и не аккумулировать их
    w.grad.data.zero_()
    b.grad.data.zero_()

    # рисуем картинки
    if (i + 1) % 5 == 0:
        clear_output(True)
        plt.figure(figsize=(10,8))
        plt.scatter(x.data.numpy(), y.data.numpy(), label="data")
        plt.scatter(x.data.numpy(), y_pred.data.numpy(),
                    color="orange", linewidth=5, label="predictions")
        plt.xlabel("LSTAT", fontsize=14)
        plt.ylabel("MEDV (target)", fontsize=14)
        plt.title("Boston modelling", fontsize=18)
        plt.legend(fontsize=14)
        plt.show()

        print("loss = ", loss.data.numpy())
        if loss.data.numpy() < 0.1:
            print("Done!")
            break


### 4. Нейросетки


![nn](nn.png)
![non_linear](active.png)
![активации](activation.png)

Для того, чтобы разобраться как обучать нейросите в pytorch, нужно освоить три вещи: 

1. Как формировать батчи и передавать их сетке
2. Как написать модель 
3. Как написать цикл обучения и отслеживать метрики

#### Dataset API

В Pytorch работа с данными строится на двух классах из [torch.utils.data](https://pytorch.org/docs/stable/data.html): `Dataset` и `DataLoader`:

- `Dataset` отвечает за подготовку одного примера
- `DataLoader` отвечает за выбор примеров, склейку их в один батч и распараллеливание на CPU, поддерживает итерирование.


Для решения задачи обычно пишут кастомные Dataset-классы, для этого нужно написать всего две функции:
- `.__len__(self)` возвращает количество примеров в датасете;
- `.__getitem__(self, item)` возвращает item-ный по счету пример из датасета.

Задачи DataLoader достаточно сложно аккуратно реализовать и лучше использовать готовый. Он довольно гибкий, все основные моменты кастомизируются заданием функций:
```
torch.utils.data.DataLoader(
    dataset,            # собственно экземпляр класса Dataset, из которого надо доставать примеры
    batch_size=1,       # количество примеров в батче
    drop_last=False,    # нужно ли при итерировании выбрасывать неполные батчи? (такое бывает, если число примеров не делится нацело на batch_size
    shuffle=False,      # перемешивать ли примеры
    sampler=None,       # чтобы перемешивать примеры кастомно
    batch_sampler=None, # чтобы использовать кастомный отбор примеров в батч
    num_workers=0,      # на сколько процессов запараллелить подготовку данных
    collate_fn=None,    # функция, которая будет склеивать примеры в батчи
    # остальные аргументы более технические, сейчас можно не рассматривать
    pin_memory=False,   
    timeout=0, 
    worker_init_fn=None, 
    multiprocessing_context=None, 
    generator=None)
```
Теперь давайте напишем такой сами, в качестве датасета сгенерируем рандомные данные.

In [ ]:
class RandomDataset(torch.utils.data.Dataset):
    """
    Our random dataset
    """
    

In [ ]:
x = np.random.rand(1000, 5)
y = np.random.rand(1000)

In [ ]:
our_dataset = RandomDataset(x, y)

Для того, чтобы из данных получать батчи в pytorch используется такая сущность как даталоадер, который принимает на вход класс унаследованный от `torch.utils.data.Dataset`. Сейчас посмотрим на пример:

In [ ]:
dataloader = torch.utils.data.DataLoader(our_dataset, batch_size=4)

Работают с ним следующим образом:

In [ ]:
batch = next(iter(dataloader))

print(f"Sample:\n{batch['sample']}")
print(f"Target:\n{batch['target']}")

#### Как сделать нейросеть

Пример как это может выглядеть:

In [ ]:
model = nn.Sequential()                   # создаем пустую модель, в которую будем добавлять слои
model.add_module("l1", nn.Linear(5, 10))  # добавили слой с 5-ю нейронами на вход и 3-мя на выход
model.add_module("l2", nn.ReLU())         # добавили функцию активации
model.add_module("l3", nn.Linear(10, 1))  # добавили слой с 3-мя нейронами на вход и 5-ю на выход

# альтернативный способ
another_model = nn.Sequential(
    nn.Linear(5, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)

In [ ]:
y_pred = model(batch['sample']) # получили предсказания модели

Может быть удобно сделать в виде класса. Нейросеть должна быть унаследована от класса `nn.Module`.

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # code
    
    def forward(self, x):
        # code

#### Как написать цикл обучения и отслеживать метрики
 
Давайте соберем загрузку данных, создание модели и обучим на мнисте.

In [ ]:
from torchvision import transforms

In [ ]:
# используем готовый класс от торча для загрузки данных для тренировки
mnist_train = torchvision.datasets.MNIST(
    "./mnist/", 
    train=True, 
    download=True, 
    transform=transforms.Compose([transforms.ToTensor()])
) 
mnist_val = torchvision.datasets.MNIST(
    "./mnist/",
    train=False, 
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

# так как это уже унаследованный от Dataset класс, его можно сразу обернуть в даталоадер
train_dataloader = torch.utils.data.DataLoader(
    mnist_train, 
    batch_size=4, 
    shuffle=True, 
    num_workers=1
) 

val_dataloader = torch.utils.data.DataLoader(
    mnist_val, 
    batch_size=4, 
    shuffle=True, 
    num_workers=1
)

In [ ]:
mnist_train[i][0].shape

In [ ]:
for i in [0, 1]:
    plt.subplot(1, 2, i + 1)
    plt.imshow(mnist_train[i][0].squeeze(0).numpy())
    plt.title(str(mnist_train[i][1]))
plt.show()

In [ ]:
model = nn.Sequential(
    nn.Flatten(),             # превращаем картинку 28х28 в вектор размером 784
    nn.Linear(28 * 28, 128),  # линейный слой, преобразующий вектор размера 784 в вектор размера 128
    nn.ReLU(),                # нелинейность
    nn.Linear(128, 10),       # линейный слой, преобразующий вектор размера 128 в вектор размера 10
)

# создаем оптимизатор, который будет обновлять веса модели
optimizer = torch.optim.SGD(model.parameters(), lr=0.05) 

Веса моделей хранятся в виде матриц и выглядят так:

In [ ]:
for x in model.named_parameters():
   print(x)

_Красиво_ трекать метрики в полуавтоматическом режиме мы будем в [wandb](https://wandb.ai). Для этого регистрируемся на сайте, устанавливаем и логинимся(это того стоит):

In [ ]:
# !pip install wandb --upgrade --quiet
import wandb

# логинимся в своего пользователя (предварительно нужно ввести ключ из настроек с wandb.ai через консоль)
wandb.login()
# инициализируем проект
wandb.init(project="pytorch-demo")
# сохраняем параметры сетки в wandb + просим следить за градиентами сетки
wandb.watch(model);

Можно перейти по ссылке и следить за нашей моделью прямо во время обучения!

In [ ]:
for epoch in range(5):
    for x_train, y_train in tqdm(train_dataloader):    # берем батч из трейн лоадера
        y_pred = model(x_train)                        # делаем предсказания
        loss = F.cross_entropy(y_pred, y_train)        # считаем лосс
        loss.backward()                                # считаем градиенты обратным проходом
        optimizer.step()                               # обновляем параметры сети
        optimizer.zero_grad()                          # обнуляем посчитанные градиенты параметров
    
    if epoch % 2 == 0:
        val_loss = []                                  # сюда будем складывать **средний по батчу** лосс
        val_accuracy = []
        with torch.no_grad():                          # на валидации запрещаем фреймворку считать градиенты по параметрам
            for x_val, y_val in tqdm(val_dataloader):  # берем батч из вал лоадера
                y_pred = model(x_val)                  # делаем предсказания
                loss = F.cross_entropy(y_pred, y_val)  # считаем лосс
                val_loss.append(loss.numpy())          # добавляем в массив 
                val_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_val).numpy().tolist())
          
        # логируем метрики
        wandb.log({"mean val loss": np.mean(val_loss),
                   "mean val accuracy": np.mean(val_accuracy)})
        
        # печатаем метрики
        print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}, accuracy: {np.mean(val_accuracy)}")

### Оптимизаторы


Чаще всего вам будут встречаться такие.


Вот [здесь](https://habr.com/ru/post/318970/) норм объяснили.



Идея методов с инерцией: «Если мы некоторое время движемся в определённом направлении, то, вероятно, нам следует туда двигаться некоторое время и в будущем».

Идея методов с масштабирование градиента: «Если мы слишком сильно обновляли какие-то параметры в прошлом, возможно, в будущем стоит обратить больше внимания на те параметры, которые мы обновляли мало».


!['modes'](modifications.png)

In [ ]:
[elem for elem in dir(torch.optim) if not elem.startswith("_")]

Основные функции PyTorch Optimizer:

- step - обновление весов модели
- zero_grad - занулить веса модели (по умолчанию градиенты в PyTorch аккумулируются) ~ ```for each param in params: param.grad = None```
- state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

#### Делаем свой Optimizer

Для того чтобы сделать свой Optimizer, не нужно писать свою имплементацию каждой из основных функций. Достаточно переопределить только одну из них - step.

Попробуем реализовать AdaGrad. 

Алгоритм следующий:

$g_t = \nabla_{w} L(w_t)$

$G_t = G_{t-1} + g_t^2$

$w_{t} = w_{t-1} - \frac{\eta}{\sqrt{G_t + \epsilon}} g_t$



В качестве данных для модели воспользуемся make_regression из sklearn.

In [ ]:
import random
import os

from sklearn.datasets import make_regression

def seed_everything(seed):
    # Зафиксировать seed.
    # Это понадобится, чтобы убедиться
    # в правильности работы нашего Optimizer
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


# make_regression возвращает 2 переменные: данные и таргет для них
# так как они возвращаётся как np.array,
# вызовем для каждого из них команду torch.from_numpy
X, y = map(
    lambda x: torch.from_numpy(x).float(),
    make_regression(n_samples=200, n_features=2)
)


def get_model():
    # Таким образом, мы при каждом вызове будем получить
    # модель с одной и той же инициализацией весов
    seed_everything(13)
    return torch.nn.Sequential(
        torch.nn.Linear(2, 10),
        torch.nn.Linear(10, 1)
    )


In [ ]:
from torch.optim import Optimizer


class InClassOptimizer(Optimizer):
    def step(self):
        """Perform a single optimization step."""
        with torch.no_grad(): # выключим градиенты
            for group in self.param_groups:
                self._group_step(group)

    def _group_step(self, group):
        # group ~ dict[str, ...]
        """
        Private helper function to perform
        single optimization step on model parameters.
        """
        raise NotImplementedError()

In [ ]:
class Adagrad(InClassOptimizer):
    def __init__(self, params, lr=0.01, eps=1e-13):
        defaults = dict(lr=lr, eps=eps)
        super().__init__(params, defaults)
        

    def _group_step(self, group):
        # One group contains information about values passed in init
        # and model parameters to update
        lr = group["lr"]
        eps = group["eps"]
        for param in filter(lambda x: x.grad is not None, group["params"]):
            pass
            # TODO:
            # Your code here
            # --------------
            # --------------

    def _get_adagrad_buffer(self, param):
        """
        Get accumulated gradients for Adagrad.

        Parameters
        ----------
        param : `torch.Tensor`, required
            Model parameter to get accumulated gradeints for Adagrad.

        Returns
        -------
        Accumulated Adagrad gradients for parameter.
        """
        param_state = self.state[param]
        
        return param_state["adagrad_buffer"]

    def _init_adagrad_buffer(self, param):
        """
        Initialize accumulated gradeints for SGD momentum.

        Parameters
        ----------
        param : `torch.Tensor`, required
            Model parameter to get accumulated gradeints for Adagrad.
        """
        param_state = self.state[param]
        if "adagrad_buffer" not in param_state:
            param_state["adagrad_buffer"] = torch.zeros_like(param)

Чекер, что все ок.

In [ ]:
def check_optimizer(model, optim, num_iter):
    loss = torch.nn.MSELoss()
    for i in range(num_iter):
        output = loss(model(X), y.unsqueeze(-1))
        output.backward()
        optim.step()
        optim.zero_grad()
        if i % 100 == 0:
            print(f"Iteration {i} loss: {output.item()}")

In [ ]:
model = get_model()
optim = Adagrad(model.parameters(), lr=0.001)
check_optimizer(model, optim, num_iter=1000)

In [ ]:
model = get_model()
optim = torch.optim.Adagrad(model.parameters(), lr=0.001)
check_optimizer(model, optim, num_iter=1000)

**Sources**: 
- https://github.com/hse-ds/iad-deep-learning/tree/master/2021/seminars
- https://github.com/m12sl/dl-hse-2021/blob/main/02-pytorch/seminar.ipynb
- https://pytorch.org/assets/deep-learning/Deep-Learning-with-PyTorch.pdf (еще больше красивых картинок)